## 태스크 2: SageMaker Processing을 사용하여 데이터 처리 수행

이 노트북에서는 Amazon SageMaker Processing을 사용하여 기본적인 Apache Spark 애플리케이션을 실행하는 데 필요한 환경을 설정합니다. SageMaker Processing에서 Apache Spark를 사용하면 Amazon EMR 클러스터를 프로비저닝하지 않고도 Spark 작업을 실행할 수 있습니다. 그런 다음 **SageMaker Python SDK**의 **PySparkProcessor** 클래스를 사용하여 Spark 작업을 정의하고 실행합니다. 그리고 마지막으로 Amazon Simple Storage Service(Amazon S3)에 저장된 데이터 처리 결과를 검증합니다.

처리 스크립트는 문자열 인덱싱, 원-핫 인코딩, 벡터 어셈블리, 처리된 데이터를 훈련 데이터세트와 검증 데이터세트로 분할(80-20 비율) 등의 몇 가지 기본적인 데이터 처리를 수행합니다.

### 태스크 2.1: 환경 설정

최신 SageMaker Python SDK 패키지와 기타 종속성을 설치합니다.

In [ ]:
%%capture
%pip install awscli --upgrade
%pip install boto3 --upgrade
%pip install -U "sagemaker=2.132.0"

SDK를 업그레이드한 후 노트북 커널을 다시 시작합니다. 

1. 노트북 도구 모음에서 **Restart kernel** 아이콘을 선택합니다.

In [ ]:
<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** You can ignore any warning you get after restarting the kernel. 


이제 필요한 라이브러리를 가져오고 SageMaker 처리 작업을 실행할 실행 역할을 가져온 다음, Spark 작업 출력을 저장할 Amazon S3 버킷을 설정합니다.


In [ ]:
#install-dependencies
import logging
import boto3
import sagemaker
import pandas as pd
from sagemaker.s3 import S3Downloader
from time import gmtime, strftime

sagemaker_logger = logging.getLogger("sagemaker")
sagemaker_logger.setLevel(logging.INFO)
sagemaker_logger.addHandler(logging.StreamHandler())

#Execution role to run the SageMaker Processing job
role = sagemaker.get_execution_role()
print("SageMaker Execution Role: ", role)

#S3 bucket to read the Spark processing script and writing processing job outputs
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        bucket = buckets.name
print("Bucket: ", bucket)

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **참고:** 오류가 발생하면 노트북 도구 모음에서 **Restart kernel** 아이콘을 선택하여 노트북 커널을 다시 시작했는지 확인합니다. 그런 다음, 셀을 재실행합니다. 그러나 위의 셀에 표시되는 경고는 무시해도 됩니다. 

### 태스크 2.2: SageMaker 처리 작업 실행

이 과제에서는 사전 처리된 데이터세트를 가져와서 검토합니다.

In [ ]:
#import-data
prefix = 'data/input'

S3Downloader.download(s3_uri=f"s3://{bucket}/{prefix}/spark_adult_data.csv", local_path= 'data/')

shape=pd.read_csv("data/spark_adult_data.csv", header=None)
shape.sample(5)

다음으로 SageMaker Spark PySparkProcessor 클래스를 생성하여 Spark 애플리케이션을 처리 작업으로 정의하고 실행합니다. 이 클래스에 관한 자세한 내용은 [SageMaker Spark PySparkProcessor](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.spark.processing.PySparkProcessor)를 참조하십시오.

PySparkProcessor 클래스를 생성할 때는 다음 파라미터를 구성합니다.
- **base_job_name**: 처리 작업 이름의 접두사
- **framework_version**: SageMaker PySpark 버전
- **role**: SageMaker 실행 역할
- **instance_count**: 처리 작업을 실행할 인스턴스의 수
- **instance_type**: 처리 작업에 사용되는 Amazon Elastic Compute Cloud(Amazon EC2) 인스턴스의 유형

In [ ]:
#pyspark-processor
from sagemaker.spark.processing import PySparkProcessor

# create a PySparkProcessor
spark_processor = PySparkProcessor(
    base_job_name="sm-spark-preprocessor",
    framework_version="3.1", # Spark version
    role=role,
    instance_count=2,
    instance_type="ml.m5.xlarge",
    max_runtime_in_seconds=1200
)

다음으로는 PySparkProcessor run 메서드를 사용하여 **pyspark_preprocessing.py** 스크립트를 처리 작업으로 실행합니다. 이 메서드에 대한 자세한 내용은 [PySparkProcessor run 메서드](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.spark.processing.PySparkProcessor.run)을 참조하십시오. 이 실습에서는 범주별 특성에 대해 문자열 인덱싱, 원-핫 인코딩 등의 데이터 변환을 수행합니다.

처리 작업을 실행하기 위해 다음 파라미터를 구성합니다.
- **submit_app**: 처리 스크립트의 경로 
- **outputs**: 사전 처리 스크립트의 출력 경로(Amazon S3 출력 위치)
- **arguments**: 사전 처리 스크립트의 명령줄 인수(예: Amazon S3 입력 및 출력 위치)

처리 작업을 완료하려면 약 5분이 소요됩니다. 작업이 실행되는 동안 파일 브라우저에서 **pyspark_preprocessing.py** 파일을 열면 이 실습의 일부분으로 미리 구성된 사전 처리 스크립트의 소스를 검토할 수 있습니다.

In [ ]:
#processing-job
import os
from sagemaker.processing import ProcessingOutput

# Amazon S3 path prefix
input_raw_data_prefix = "data/input"
output_preprocessed_data_prefix = "data/output"
logs_prefix = "logs"

# Run the processing job
spark_processor.run(
    submit_app="pyspark_preprocessing.py",
    outputs=[
        ProcessingOutput(output_name="train_data", 
                         source="/opt/ml/processing/train",
                         destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "train")),
        ProcessingOutput(output_name="validation_data", 
                         source="/opt/ml/processing/validation",
                         destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "validation")),
    ],
    arguments=[
        "--s3_input_bucket", bucket,
        "--s3_input_key_prefix", input_raw_data_prefix,
        "--s3_output_bucket", bucket,
        "--s3_output_key_prefix", output_preprocessed_data_prefix],
    spark_event_logs_s3_uri="s3://{}/{}/spark_event_logs".format(bucket, logs_prefix),
    logs=True
)

print("Spark Processing Job Completed.")

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **참고:** 처리 작업 실행 중에 SageMaker AI 콘솔에서 작업 진행 상황을 모니터링할 수도 있습니다. 처리 작업을 모니터링하려면 다음 단계를 수행합니다. 

1. SageMaker AI 콘솔로 이동합니다. 

2. 왼쪽 창에서 **Processing**을 선택하고 **Processing jobs**를 선택합니다. 

3. 처리 작업 이름은 **sm-spark-preprocessor-**로 시작됩니다. 

4. 처리 작업이 완료되면 이 노트북으로 돌아옵니다. 

### 태스크 2.3: 데이터 처리 결과 검증

실행한 데이터 처리 작업의 출력을 검증합니다. 이렇게 하려면 훈련 및 검증 출력 데이터세트의 첫 5개 행을 검토합니다.

In [ ]:
#view-train-dataset
print("Top 5 rows from s3://{}/{}/train/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/train/train_features.csv - | head -n5

In [ ]:
#view-validation-dataset
print("Top 5 rows from s3://{}/{}/validation/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/validation/validation_features.csv - | head -n5

### 결론

축하합니다! SageMaker Processing에서 SageMaker Python SDK를 사용하여 Spark 처리 작업을 생성했으며 처리 작업을 실행했습니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 노트북 파일을 닫습니다.
- 실습 세션으로 돌아가 **결론**을 계속 진행합니다.